
**Introduction to My Solution**
1. **Data Loading and Module Setup**: Initially, I load the required data and modules.
2. **Table Extraction**: I extract tables from PDFs into pandas DataFrame format.
3. **Data Preparation**: The extracted tables are processed and prepared for subsequent steps.
4. **2022 Metric Value Extraction**: Leveraging values from 2019, 2020, and 2021, alongside similarity models, I extract the 2022 values of certain metrics.
5. **LLM Model Training**: The extracted data is used to train Language Model (LLM) models capable of generating values for the remaining metrics.

# Data and modules

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import os
import math
import re
from collections import Counter


In [ ]:
!pip install camelot-py
!pip install sentence-transformers
import camelot
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, InputExample, losses,util
model_name="Sakil/sentence_similarity_semantic_search"
model = SentenceTransformer(model_name)

In [3]:
root_dir = '/content/drive/MyDrive/Data/report_comp'
pdf_paths = []
for foldername, subfolders, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(foldername, filename)
            pdf_paths.append(pdf_path)

In [10]:
train=pd.read_csv('/content/drive/MyDrive/Data/report_comp/Train (5).csv')
AMKEY=pd.read_csv('/content/drive/MyDrive/Data/report_comp/AMKEY_GoldenStandard.csv')
Act=pd.read_csv('/content/drive/MyDrive/Data/report_comp/ActivityMetricsSynonyms.csv')
PDF=pd.read_csv('/content/drive/MyDrive/Data/report_comp/PDFnames.csv')
sub=pd.read_csv('/content/drive/MyDrive/Data/report_comp/SampleSubmission (11).csv')

In [ ]:
sub['AMKEY']=[int(x.split('_')[0]) for x in sub.ID]
sub['Group']=[x.split('_')[-1] for x in sub.ID]


In [ ]:
# This cell reads PDF files using Camelot, a Python library for extracting tables from PDFs.
# D_pdf have ,for each pdf, a list of pd.DataFrame extracted tables
D_pdf={}
for i in tqdm(range(len(pdf_paths))):
  name=PDF.loc[PDF['File Name']==pdf_paths[i].split('/')[-1]].iloc[0,0].split('.')[0]
  pdf1_C = camelot.read_pdf(pdf_paths[i], flavor='stream', pages='all')
  D_pdf[name]=pdf1_C

# Tables processing

In [3]:
# Extracts numerical values from a given text.
def extract_digits(text):
    pattern = r'\d{1,10}(?: \d{10})*(?:,\d+)?(?:\.\d+)?'
    matches = re.findall(pattern, text.replace(' ', '').replace(',', '.'))
    if matches:
        return matches[0]
    else:
        return 0
# ------------------------------
# Checks if any columns in the provided DataFrame contain convertible values.

def has_convertible_column(table):
  M=[]
  for col in table.columns:
    J=[]
    for i in range(table.shape[0]):
      try:
        J+=[float(table[col][i].replace(' ','').replace('R',''))]
      except ValueError:
          pass
    M+=[len(J)]
  for c in range(len(table.columns)):
    if M[c]>2:
      result_list = [extract_digits(x) for x in list(table[table.columns[c]])]
      table[table.columns[c]]=result_list
  T=table.copy()
  return T
# ------------------------------
# Replaces null or zero values in the DataFrame with values from subsequent rows.

def Rp_Z(df):
    df0=df.copy()
    for i in range(len(df0)):
        num_zeros_null_empty = sum(pd.isnull(df0.iloc[i, j]) or df0.iloc[i, j] == 0 or df0.iloc[i, j] == '' for j in range(len(df.columns)))
        if num_zeros_null_empty >= 2:
            for j in range(len(df0.columns)):
                if pd.isnull(df0.iloc[i, j]) or df0.iloc[i, j] == 0 or df0.iloc[i, j] == '':
                    if i < len(df) - 1:
                        df0.iloc[i, j] = df0.iloc[i + 1, j]
        else:
            for j in range(len(df0.columns)):
                if pd.isnull(df0.iloc[i, j]) or df0.iloc[i, j] == '':
                    df0.iloc[i, j] = 0
    return df0

In [9]:
# Eliminate tables without numeric values and perform replacement of null or zero values.
D_tabs={}
for j in D_pdf.keys():
  T_tabs=[]
  tab=D_pdf[j]
  for i in range(len(tab)):
    D=tab[i].df.copy()
    if (tab[i].df.equals(has_convertible_column(D))) ==False:
      T_tabs+=[D]
  D_tabs[j]=T_tabs
D_tabs0={}
for i in tqdm(D_tabs.keys()):
  D_tabs0[i]=[Rp_Z(D_tabs[i][j]) for j in range(len((D_tabs[i])))]
D_tabs=D_tabs0

100%|██████████| 12/12 [00:17<00:00,  1.48s/it]


In [ ]:
# Append the values of tables 'Uct1' and 'Uct2' to create a new table 'Uct1&2' in D_tabs and remove the old tables.
# Same for Oceana
D_tabs['Uct1&2']=D_tabs['Uct1']+D_tabs['Uct2']
D_tabs['Oceana1&2']=D_tabs['Oceana1']+D_tabs['Oceana2']
D_tabs.pop('Uct1')
D_tabs.pop('Uct2')
D_tabs.pop('Oceana1')
D_tabs.pop('Oceana2')

In [11]:
def metric_col(tab,L_tab):
  L=[]
  for col in tab.columns:
    l=0
    for j in L_tab:
      if j in list(tab[col]):
        l+=1
    L+=[l]
  return L
def pos(L,o=0):
  p=0
  for i in range(o,len(L)):
    if L[i]>0:return i
    else:p+=1
  if p-o ==len(L):
    return 0
def tab2metriccols(tabs,L_tab):
  ind0=[0]
  ind=[]
  i=-1
  DT={}
  for tab in tabs:
    i+=1
    if len(metric_col(tab,L_tab))>4:
      ind0=[0,pos(metric_col(tab,L_tab),1)]
      if ind0[-1]==None:
        ind0=[ind0[0]]
      DT[i]=ind0
    else:
      ind=[pos(metric_col(tab,L_tab),0)]
      DT[i]=ind
  return DT

In [12]:
train['AMKEY']=[int(x.split('_')[0]) for x in train.ID]
train['Group']=[x.split('_')[-1] for x in train.ID]

# First Predictions

**With the table extracted using Camelot and the known values of the metrics in 2019, 2020, and 2021,**

**this section can infer the values of certain metrics in 2022. This information will be valuable for training the LLM  to extract the 2022 values of the remaining metrics.**

In [13]:
K=[]
Tb={}
A={}
#This loop iterates over each company and uses the tables extracted from Camelot to retrieve the values of 2022. It achieves this by employing similarity models to find matching patterns and extract the relevant information.

for name in tqdm(D_tabs.keys()):
  L_tab=list(Act.loc[Act.Group==name].loc[Act.AMKEY.isin(list(train.loc[train['2021_Value'].isnull()==False].loc[train.Group==name].AMKEY))].ClientMetric)
  LL=False
  if L_tab==[]:
    LL=True
    L_tab=list(AMKEY.loc[AMKEY.AMKEY.isin(list(train.loc[(train.Group==name) & (train['2021_Value'].isnull()==False)]['AMKEY']))].ActivityMetric)
  K=[]
  M=[]
  DT=tab2metriccols(D_tabs[name],L_tab)
  Tb0={}
  for j in range(len(D_tabs[name])):
    ind=DT[j]
    for k in ind:
      M+=list(D_tabs[name][j].iloc[:,k])
    for l in L_tab:
      for k in ind:
        if (l in list(D_tabs[name][j].iloc[:,k])):
          K+=[l]
  K=list(set(K))
  #NF the metrics that are not found in the tables
  NF=[k for k in L_tab  if k not in K ]
  embeddings_list1 = model.encode(NF, convert_to_tensor=True)
  M0 = ['' if isinstance(sentence, float) and np.isnan(sentence) else sentence for sentence in M]
  M0 = [str(item) for item in M0 if item is not None]
  embeddings_list2 = model.encode(M0, convert_to_tensor=True)
    # Calculate cosine similarity
  cosine_scores = util.pytorch_cos_sim(embeddings_list1, embeddings_list2)
    # Find the most similar sentence for unfound metrics
  most_similar_indices = cosine_scores.argmax(dim=1).tolist()
  most_similar_sentences = [M0[index] for index in most_similar_indices]
    #MM the all the elements that can extract the values
  Rep_D={}
  for i,j in zip(NF,most_similar_sentences):
    Rep_D[i]=j
  MM= K+most_similar_sentences
  Df=[]
  tb0=[]
  for m in MM:
    p=0
    for T in D_tabs[name]:
      K+=[T]
      ind=DT[p]
      if len(ind)>1:
        for k in range(len(ind)):
          if T.loc[T[T.columns[ind[k]]]==m].shape[0]>0:
            tb0+=[list(T.loc[T[T.columns[ind[k]]]==m])]
            if ind.index(ind[k])>0:
              Df+=[list(T.loc[T[T.columns[ind[k]]]==m].iloc[0,ind[k]:])]
            else:
              Df+=[list(T.loc[T[T.columns[ind[k]]]==m].iloc[0,ind[k]:ind[k+1]])]
      else:
        if T.loc[T[T.columns[ind[0]]]==m].shape[0]>0:
          Df+=[list(T.loc[T[T.columns[ind[0]]]==m].iloc[0,ind[0]:])]
      p+=1
  Df0=[dd[:6] + (6-len(dd))*[0] for dd in Df]
  TT=pd.DataFrame(Df0,columns=['col'+str(i) for i in range(6)])
  train['AMKEY']=[int(x.split('_')[0]) for x in train.ID]
  train['Group']=[x.split('_')[-1] for x in train.ID]
  tr=train.loc[train.Group==str(name)].loc[train['2021_Value'].isnull()==False]
  if LL==True:
    tr0=AMKEY.loc[AMKEY.AMKEY.isin(list(train.loc[(train.Group==name) & (train['2021_Value'].isnull()==False)]['AMKEY']))][['AMKEY','ActivityMetric']].merge(tr ,on=['AMKEY'],how='left')
    tr1=tr0.loc[tr0.Group==name]
  else:
    tr0=Act.loc[Act.Group==name].loc[Act.AMKEY.isin(list(train.loc[train['2021_Value'].isnull()==False].loc[train.Group==name].AMKEY))][['AMKEY','Group','ClientMetric']].merge(tr ,on=['AMKEY','Group'],how='left')
    tr1=tr0.loc[tr0.Group==name]
  for i in NF:
    TT['col0']=TT['col0'].replace(Rep_D[i],i)
  TT0=TT.rename(columns={'col0':'ClientMetric'})
  A0=tr1.rename(columns={'ActivityMetric':'ClientMetric'}).merge(TT0,on='ClientMetric',how='left')
  A0.fillna(0, inplace=True)
  A0.replace('', 0, inplace=True)
  ar0=np.array(A0[['2021_Value','2020_Value']])
  ar1=np.array(A0.iloc[:,7:])
  index=[]
  Tr1=[]
  Ii=[]
  for i in range(len(ar0)):
    P=ar0[i]
    P0=ar1[i]
    P=[str(item) for item in P]
    embeddings_list1 = model.encode(P, convert_to_tensor=True)
    M0=[str(item) for item in P0]
    embeddings_list2 = model.encode(M0, convert_to_tensor=True)
    # Calculate cosine similarity
    cosine_scores = util.pytorch_cos_sim(embeddings_list1, embeddings_list2)
    # Find the most similar sentence for  the values
    most_similar_indices = cosine_scores.argmax(dim=1).tolist()
    most_similar_sentences = [M0[index] for index in most_similar_indices]
    P0=[str(p) for p in P0]
    second_array = np.array(P0)
    first_array = np.array(most_similar_sentences)
    # Find the index of the first list in the second list
    index = next((i for i in range(len(second_array) - len(first_array) + 1) if np.array_equal(second_array[i:i+len(first_array)], first_array)), 0)
    Ii+=[index]
    if index==0:
      second_array=np.array(list(['0']+list(second_array)))
      index = next((i for i in range(len(second_array) - len(first_array) + 1) if np.array_equal(second_array[i:i+len(first_array)], first_array)), 0)
    if index>0:
      try :
        m=float(P[0])/float(extract_digits(str(most_similar_sentences[0])))
      except:
        m=1
        pass
      Tr1+=[float(extract_digits(P0[index-1]))*m]
    else:Tr1+=['0']
  A0['2022_Value']=Tr1
  A0['ind']=Ii
  A[name]=A0
  for id in A0.ID:
    sub.iloc[sub.loc[sub.ID==id].index[0],1]=float(list(A0.loc[A0.ID==id]['2022_Value'])[0])

100%|██████████| 10/10 [05:40<00:00, 34.01s/it]


In [14]:
As=pd.DataFrame([[0]*len(A['Absa'].columns)],columns=A['Absa'].columns).drop(0)
for k in A.keys():
  As=pd.concat([As,A[k]])

In [15]:
sub.to_csv('sub0.csv',index=False)

# LLM Model

In [ ]:
!pip install -U sentence-transformers --quiet
!pip install -U transformers --quiet
!pip install accelerate --quiet
%pip install llama-index-llms-huggingface --quiet
!pip install chromadb --quiet
!pip install datasets --quiet
!pip install langchain --quiet
!pip install llama-index
!pip install datasets
!pip install llama-parse
!pip install llama-index-vector_stores-chroma

!pip install llama-index-embeddings-langchain

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse
key='llx-J1us2HmL1JUcFImdNvnwiktMruzL5ncEYLee6IcmSxVtB3pk'
parser = LlamaParse(api_key=key ,result_type="text")
# sync
documents = parser.load_data(pdf_paths[:3])

In [ ]:
pdf_paths

In [20]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import ServiceContext
from sentence_transformers import SentenceTransformer
import chromadb
# Load a Huggingface embedding model
embed_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# set up ChromaVectorStore and load in data create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("gemma7b-test")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/chromadb/segment/impl/vector/local_hnsw.py:26: RuntimeWarning: coroutine 'LlamaParse._aload_data' was never awaited
  import hnswlib


In [ ]:
from llama_index.core.node_parser import SentenceSplitter

base_splitter = SentenceSplitter(chunk_size=75,chunk_overlap=6)
nodes = base_splitter.get_nodes_from_documents(documents)
index = VectorStoreIndex(nodes,storage_context=storage_context, embed_model=embed_model)

In [ ]:
As['2022_Value']=As['2022_Value'].astype(float)
As0=As.loc[As['2022_Value']>0].reset_index().drop('index',axis=1)
As1=As.loc[As['2022_Value']==0].reset_index().drop('index',axis=1)
As0=As0.loc[As0.Group!='Tongaat'].loc[As0.Group!='Oceana1&2']
As1

In [ ]:
Values1, Questions1, Context1, ID1, Gp1, Y1 = [], [], [], [], [], []
Values, Questions, Context, ID0, Gp0, Y0 = [], [], [], [], [], []

#extract the context from the table As

for i in tqdm(range(As0.shape[0])):
  k=0
  for year in ['2019','2020','2021','2022']:
    ID0 += [As1.ID[i]]
    Gp0 += [As1.Group[i]]
    Y0 += [year]
    Values+=[As0[['2019_Value','2020_Value','2021_Value','2022_Value']].iloc[i,k]]
    k+=1
    Questions+=['the value of '+list(As0['ClientMetric'])[i] + ' in '+ list(As0['Group'])[i] +' document in '+year +'?']
    Context+=[' || '.join(str(As0.iloc[i,9:-2][x]) for x in As0.iloc[i,9:-2].keys())]
for i in tqdm(range(As1.shape[0])):
  k=0
  for year in ['2019','2020','2021']:
    ID1 += [As1.ID[i]]
    Gp1+= [As1.Group[i]]
    Y1+= [year]
    Values1+=[As1[['2019_Value','2020_Value','2021_Value']].iloc[i,k]]
    k+=1
    Questions1+=['the value of '+list(As1['ClientMetric'])[i] + ' in '+ list(As1['Group'])[i] +' document in '+year +'?']
    Context1+=[' || '.join(str(As1.iloc[i,9:-2][x]) for x in As1.iloc[i,9:-2].keys())]

Q_tot=Questions+Questions1
V_tot=Values+Values1
C_tot=Context+Context1

#extract the context using RAG approch
C_00=[]
for q in tqdm(Q_tot):
  retriever = index.as_retriever(similarity_top_k=3)
  results = retriever.retrieve(q)
  C_00 += ['  \n  '.join(j.text for j in results)]

In [32]:
#this cell built a train dataset containing the questions,the contexts,and the values of the metrics

data0=pd.DataFrame([['0']*3],columns=['Question','Context','CC']).drop(0)
data0['Question']=Q_tot
data0['CC']=C_tot
data0['Context']=''
data0['Value']=V_tot

V00,V11=[],[]

for i in tqdm(range(As0.shape[0])):
  k=0
  v1=[]
  for year in ['2019','2020','2021']:
    v1+=[As0[['2019_Value','2020_Value','2021_Value']].iloc[i,k]]
    k+=1
  V00+=[v1]*4
for i in tqdm(range(As1.shape[0])):
  k=0
  v1=[]
  for year in ['2019','2020','2021']:
    v1+=[As1[['2019_Value','2020_Value','2021_Value']].iloc[i,k]]
    k+=1
  V11+=[v1]*3
V=V00+V11

Data0=pd.concat([data0,pd.DataFrame(V,columns=['2019_Value','2020_Value','2021_Value'])],axis=1)
Data0['year']=[2019,2020,2021,2022]*As0.shape[0]+[2019,2020,2021]*As1.shape[0]

In [40]:
Q22=[]
C22=[]
Ids=[]
#extract the context from the table As
for i in tqdm(range(As1.shape[0])):
  Ids+=[As1.iloc[i,3]]
  Q22+=['the value of '+list(As1['ClientMetric'])[i] + 'in '+ list(As1['Group'])[i] +' document in 2022 ?']
  C22+=[' || '.join(str(As1.iloc[i,9:-2][x]) for x in As1.iloc[i,9:-2].keys())]
#extract the context using RAG approch
C_10=[]
for q in tqdm(Q22):
  retriever = index.as_retriever(similarity_top_k=3)
  results = retriever.retrieve(q)
  C_10 +=['  \n  '.join(j.text for j in results)]

100%|██████████| 422/422 [00:00<00:00, 869.17it/s]


In [ ]:
As1

In [41]:
#this cell built a test dataset containing the questions,the contexts,and the values of the metrics
data1=pd.DataFrame([['0']*3],columns=['Question','Context','CC']).drop(0)
data1['Question']=Q22
data1['CC']=C22
data1['Context']=C_10
data1['ID']=Ids
V01=[]
for i in tqdm(range(As1.shape[0])):
  k=0
  v1=[]
  for year in ['2019','2020','2021']:
    v1+=[As1[['2019_Value','2020_Value','2021_Value']].iloc[i,k]]
    k+=1
  V01+=[v1]
Data1=pd.concat([data1,pd.DataFrame(V01,columns=['2019_Value','2020_Value','2021_Value'])],axis=1)
Data1['year']='2022'

100%|██████████| 422/422 [00:01<00:00, 334.85it/s]


In [42]:
#generate the prompt for each question
def generate_prompt(data_point):
    prefix_text =f"""Given the question and context, extract the value of the metric in {data_point["year"]} .\n\n"""
    if data_point["year"]==2022:
        Vly=f"""[INST]  the value of the same metric in 2019 is  {data_point["2019_Value"]} [/INST]\n  [INST]  the value of the same metric in 2020 is  {data_point["2020_Value"]} [/INST]\n [INST]  the value of the same metric in 2021 is  {data_point["2021_Value"]} [/INST]\n  """
    elif data_point["year"]==2021:
        Vly=f"""[INST]  the value of the same metric in 2019 is  {data_point["2019_Value"]} [/INST]\n  [INST]  the value of the same metric in 2020 is  {data_point["2020_Value"]} [/INST]\n"""
    elif data_point["year"]==2020:
        Vly=f"""[INST]  the value of the same metric in 2019 is  {data_point["2019_Value"]} [/INST]\n"""
    else:Vly=''
    text = f"""[INST] {prefix_text}Question: {data_point["Question"]}[/INST] [INST]Context: {data_point["Context"]}  \n  {data_point["CC"]} \n [/INST]\n [INST] {Vly} [/INST]\n [INST]The result value for the metric : {data_point["Value"]}[/INST]\n"""
    return text
# add the "prompt" column in the dataset
text_column = [generate_prompt(Data0.iloc[i]) for i in range(Data0.shape[0])]
Data0["prompt"]=text_column

In [43]:
def generate_prompt(data_point):
    prefix_text =f"""Given the question and context, extract the value of the metric in {data_point["year"]} .\n\n"""
    Vly=f"""[INST]  the value of the same metric in 2019 is  {data_point["2019_Value"]} [/INST]\n  [INST]  the value of the same metric in 2020 is  {data_point["2020_Value"]} [/INST]\n [INST]  the value of the same metric in 2021 is  {data_point["2021_Value"]} [/INST]\n  """
    text = f"""[INST] {prefix_text}Question: {data_point["Question"]}[/INST] [INST]Context: {data_point["Context"]}  \n  {data_point["CC"]} \n [/INST]\n [INST] {Vly} [/INST]\n [INST]The result value for the metric : [/INST]\n"""
    return text
# add the "prompt" column in the dataset
text_column = [generate_prompt(Data1.iloc[i]) for i in range(Data1.shape[0])]
Data1["prompt"]=text_column

**For matters of time and resources, I didn't include the training cell. Instead, I loaded the pre-trained model, which has already been trained on the training data (Data0). Therefore, I will directly use the pipeline from HuggingFace to load the model.**

In [1]:
!pip install -q  torch  datasets

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline)


In [ ]:
## Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "llama_prompt"
# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-generation", model="StaAhmed/llama_prompt")

In [5]:
Data1=Data1.drop_duplicates('ID').reset_index().drop('index',axis=1)

In [ ]:
O=[]
for j in tqdm(range(Data1.shape[0])):
    O+=[pipe(Data1['prompt'][j],max_new_tokens=150)]

In [ ]:
E=[]
for r in O:
  E+=[extract_digits(r[0]['generated_text'].split('The result value for the metric')[-1].split('2022')[-1])]
Data1['2022_Value']=E
Data1['2022_Value']=Data1['2022_Value'].astype(float)

In [ ]:
R=[]
for j in tqdm(sub.ID):
    if j in list(Data1.ID):
        R+=[list(Data1.loc[Data1.ID==j]['2022_Value'])[0]]
    elif list(sub.loc[sub.ID==j]['2022_Value'])[0]>0:
        R+=[list(sub.loc[sub.ID==j]['2022_Value'])[0]]
    else :R+=[0]
R

In [ ]:
sub['2022_Value']=R
sub.to_csv('Final_submission',index=False)